# Credit Card Approval Case Study

In [36]:
import os
import sys
# Here you need to have same Python version on your local machine and on worker node i.e. EC2. here both should have python3.
os.environ["PYSPARK_PYTHON"] = "/bin/python3"
os.environ["JAVA_HOME"] = "/usr/java/jdk1.8.0_161/jre"
os.environ["SPARK_HOME"] = "/home/ec2-user/spark-2.4.4-bin-hadoop2.7"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
sys.path.insert(0, os.environ["PYLIB"] + "/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] + "/pyspark.zip")

In [37]:

#Importing all necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt 
from pyspark.sql.functions import *
import math

#Creating Spark Session
from pyspark.sql import *
spark = SparkSession.builder.appName('CreditCard_Model').getOrCreate()

## Step 1: Importing and Merging Data

In [38]:
#Reading Data from a CSV file 
#Inferring Schema and Setting Header as True
df1=spark.read.csv('application_record.csv', header=True, inferSchema=True)
#Printing Schema
df1.printSchema()

[Stage 44:=============================>                            (1 + 1) / 2]

root
 |-- ID: integer (nullable = true)
 |-- CODE_GENDER: string (nullable = true)
 |-- FLAG_OWN_CAR: string (nullable = true)
 |-- FLAG_OWN_REALTY: string (nullable = true)
 |-- CNT_CHILDREN: integer (nullable = true)
 |-- AMT_INCOME_TOTAL: double (nullable = true)
 |-- NAME_INCOME_TYPE: string (nullable = true)
 |-- NAME_EDUCATION_TYPE: string (nullable = true)
 |-- NAME_FAMILY_STATUS: string (nullable = true)
 |-- NAME_HOUSING_TYPE: string (nullable = true)
 |-- DAYS_BIRTH: integer (nullable = true)
 |-- DAYS_EMPLOYED: integer (nullable = true)
 |-- FLAG_MOBIL: integer (nullable = true)
 |-- FLAG_WORK_PHONE: integer (nullable = true)
 |-- FLAG_PHONE: integer (nullable = true)
 |-- FLAG_EMAIL: integer (nullable = true)
 |-- OCCUPATION_TYPE: string (nullable = true)
 |-- CNT_FAM_MEMBERS: double (nullable = true)



As we can see the variable CNT_FAM_MEMBER is given as double integer. it is clear that number of people cannot be a decimal or float number. So the datatype need to be cast as an integer.

In [39]:
#Convert the datatype of the column
#Casting CNT_FAM_MEMBERS into integer
from pyspark.sql.types import * 
df1 = df1.withColumn('CNT_FAM_MEMBERS', df1['CNT_FAM_MEMBERS'].cast(IntegerType()))

In [40]:
#Reading Data from a CSV file 
#Inferring Schema and Setting Header as True
df2=spark.read.csv('credit_record.csv', header=True, inferSchema=True)
#Printing Schema
df2.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- MONTHS_BALANCE: integer (nullable = true)
 |-- STATUS: string (nullable = true)



In [41]:
#Data samples
df1.show(2)

+-------+-----------+------------+---------------+------------+----------------+----------------+-------------------+------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+
|     ID|CODE_GENDER|FLAG_OWN_CAR|FLAG_OWN_REALTY|CNT_CHILDREN|AMT_INCOME_TOTAL|NAME_INCOME_TYPE|NAME_EDUCATION_TYPE|NAME_FAMILY_STATUS|NAME_HOUSING_TYPE|DAYS_BIRTH|DAYS_EMPLOYED|FLAG_MOBIL|FLAG_WORK_PHONE|FLAG_PHONE|FLAG_EMAIL|OCCUPATION_TYPE|CNT_FAM_MEMBERS|
+-------+-----------+------------+---------------+------------+----------------+----------------+-------------------+------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+
|5008804|          M|           Y|              Y|           0|        427500.0|         Working|   Higher education|    Civil marriage| Rented apartment|    -12005|        -4542|         1|              1|         0|

In [42]:
#Data samples
df2.show(5)

+-------+--------------+------+
|     ID|MONTHS_BALANCE|STATUS|
+-------+--------------+------+
|5001711|             0|     X|
|5001711|            -1|     0|
|5001711|            -2|     0|
|5001711|            -3|     0|
|5001712|             0|     C|
+-------+--------------+------+
only showing top 5 rows



In [43]:
#dropping the column, since its only the months in which the data is extracted
df2=df2.drop("MONTHS_BALANCE")

In [44]:
df2.groupBy(df2.columns).count().where(col('count') > 1).select(sum('count')).show()

[Stage 49:=============================>                            (1 + 1) / 2]

+----------+
|sum(count)|
+----------+
|   1031869|
+----------+



In [45]:
#dropping duplicates
df2=df2.dropDuplicates()

In [46]:
df2.groupBy(df2.columns).count().where(col('count') > 1).select(sum('count')).show()

[Stage 53:============================================>         (164 + 2) / 200]

+----------+
|sum(count)|
+----------+
|      null|
+----------+



### Combining all data files into one consolidated dataframe

In [47]:
#merging the dataframes
data=df1.join(df2, on="ID", how="inner")

## Step 2: Inspecting the Dataframe

In [48]:
#data samples
data.show(3)

[Stage 57:================================================>     (181 + 2) / 200]

+-------+-----------+------------+---------------+------------+----------------+----------------+--------------------+------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+------+
|     ID|CODE_GENDER|FLAG_OWN_CAR|FLAG_OWN_REALTY|CNT_CHILDREN|AMT_INCOME_TOTAL|NAME_INCOME_TYPE| NAME_EDUCATION_TYPE|NAME_FAMILY_STATUS|NAME_HOUSING_TYPE|DAYS_BIRTH|DAYS_EMPLOYED|FLAG_MOBIL|FLAG_WORK_PHONE|FLAG_PHONE|FLAG_EMAIL|OCCUPATION_TYPE|CNT_FAM_MEMBERS|STATUS|
+-------+-----------+------------+---------------+------------+----------------+----------------+--------------------+------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+------+
|5009033|          F|           N|              N|           0|        255150.0|       Pensioner|   Incomplete higher|    Civil marriage| Rented apartment|    -18682|       365243|         1|  

In [49]:
#Statistical summary
data.describe().show()

[Stage 62:====================================================> (194 + 2) / 200]

+-------+-----------------+-----------+------------+---------------+-------------------+------------------+--------------------+--------------------+------------------+-----------------+------------------+------------------+----------+-------------------+------------------+-------------------+--------------------+------------------+-------------------+
|summary|               ID|CODE_GENDER|FLAG_OWN_CAR|FLAG_OWN_REALTY|       CNT_CHILDREN|  AMT_INCOME_TOTAL|    NAME_INCOME_TYPE| NAME_EDUCATION_TYPE|NAME_FAMILY_STATUS|NAME_HOUSING_TYPE|        DAYS_BIRTH|     DAYS_EMPLOYED|FLAG_MOBIL|    FLAG_WORK_PHONE|        FLAG_PHONE|         FLAG_EMAIL|     OCCUPATION_TYPE|   CNT_FAM_MEMBERS|             STATUS|
+-------+-----------------+-----------+------------+---------------+-------------------+------------------+--------------------+--------------------+------------------+-----------------+------------------+------------------+----------+-------------------+------------------+----------------

### Checking for duplicate rows

In [50]:
#Checking for duplicate rows
data.groupBy(data.columns).count().where(col('count') > 1).select(sum('count')).show()

[Stage 67:=================================================>    (184 + 2) / 200]

+----------+
|sum(count)|
+----------+
|      null|
+----------+



### Handling Missing and Null values

In [51]:
# Get count of nan or missing values
data.select([count(when(isnan(c), c)).alias(c) for c in data.columns]).show()

[Stage 72:=================================================>    (184 + 2) / 200]

+---+-----------+------------+---------------+------------+----------------+----------------+-------------------+------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+------+
| ID|CODE_GENDER|FLAG_OWN_CAR|FLAG_OWN_REALTY|CNT_CHILDREN|AMT_INCOME_TOTAL|NAME_INCOME_TYPE|NAME_EDUCATION_TYPE|NAME_FAMILY_STATUS|NAME_HOUSING_TYPE|DAYS_BIRTH|DAYS_EMPLOYED|FLAG_MOBIL|FLAG_WORK_PHONE|FLAG_PHONE|FLAG_EMAIL|OCCUPATION_TYPE|CNT_FAM_MEMBERS|STATUS|
+---+-----------+------------+---------------+------------+----------------+----------------+-------------------+------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+------+
|  0|          0|           0|              0|           0|               0|               0|                  0|                 0|                0|         0|            0|         0|              0|      

There are no missing values in dataset

In [52]:
### Get count of null values
data.select([count(when(col(c).isNull(), c)).alias(c) for c in data.columns]).show()

[Stage 77:===========================================>          (161 + 2) / 200]

+---+-----------+------------+---------------+------------+----------------+----------------+-------------------+------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+------+
| ID|CODE_GENDER|FLAG_OWN_CAR|FLAG_OWN_REALTY|CNT_CHILDREN|AMT_INCOME_TOTAL|NAME_INCOME_TYPE|NAME_EDUCATION_TYPE|NAME_FAMILY_STATUS|NAME_HOUSING_TYPE|DAYS_BIRTH|DAYS_EMPLOYED|FLAG_MOBIL|FLAG_WORK_PHONE|FLAG_PHONE|FLAG_EMAIL|OCCUPATION_TYPE|CNT_FAM_MEMBERS|STATUS|
+---+-----------+------------+---------------+------------+----------------+----------------+-------------------+------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+------+
|  0|          0|           0|              0|           0|               0|               0|                  0|                 0|                0|         0|            0|         0|              0|      

There is null values in column OCCUPATION_TYPE of data. This need to be imputed

Null values in OCCUPATION_TYPE only points to people who didnt mentioned it. Either their proffesion was not mentioned in the options available or they didnt want to divulge it. These null values are mapped to a new variable "Not Mentioned", and thus handled.

In [53]:
# replace the null values in the columns "OCCUPATION_TYPE" with 'Not mentioned' using the withColumn and when functions
data = data.withColumn("OCCUPATION_TYPE",when(data["OCCUPATION_TYPE"].isNull(), "Not Mentioned").otherwise(data["OCCUPATION_TYPE"]))

In [54]:
#checking if imputed properly
data.groupby(["OCCUPATION_TYPE"]).count().sort(desc("count")).show()

[Stage 81:=============================>                            (1 + 1) / 2]

+--------------------+-----+
|     OCCUPATION_TYPE|count|
+--------------------+-----+
|       Not Mentioned|23264|
|            Laborers|12621|
|          Core staff| 7501|
|         Sales staff| 7048|
|            Managers| 6246|
|             Drivers| 4404|
|High skill tech s...| 2918|
|         Accountants| 2544|
|      Medicine staff| 2498|
|       Cooking staff| 1335|
|      Security staff| 1232|
|      Cleaning staff| 1126|
|Private service s...|  659|
|  Low-skill Laborers|  370|
|Waiters/barmen staff|  361|
|         Secretaries|  314|
|            HR staff|  165|
|       Realty agents|  146|
|            IT staff|  122|
+--------------------+-----+



## Step 3: Outlier treatment and variable encoding and transformation

Before looking for outliers variables DAYS_BIRTH and DAYS_EMPLOYED need to be transformed properly. They are given as negative values mostly since they are calcualted from current date and backwards. These variables are transformed into AGE and YEARS_EMPLOYED since they are more meaningful and insightful.

In [55]:
#Creating new varaibles to calculate age and employment years
data=data.withColumn("AGE", data["DAYS_BIRTH"]/-365.25)
data=data.withColumn("YEARS_EMPLOYED", data["DAYS_EMPLOYED"]/-365.25)

#casting proper data types
data=data.withColumn("AGE", data["AGE"].cast(IntegerType()))
data=data.withColumn("YEARS_EMPLOYED", data["YEARS_EMPLOYED"].cast(IntegerType()))

#dropping columns
data = data.drop('DAYS_BIRTH', 'DAYS_EMPLOYED')

In [56]:
#Creating new variable monthly income
data=data.withColumn("MONTHLY_INCOME", data["AMT_INCOME_TOTAL"]/12)

#dropping columns
data = data.drop("AMT_INCOME_TOTAL")

In [57]:
data.groupby(["STATUS"]).count().sort(desc("count")).show()

[Stage 85:==================================================>   (188 + 2) / 200]

+------+-----+
|STATUS|count|
+------+-----+
|     0|31963|
|     X|19648|
|     C|17950|
|     1| 4149|
|     2|  555|
|     3|  248|
|     4|  181|
|     5|  180|
+------+-----+



In [58]:
#replacing vales of X and C from Status column to -1 and -2 
data=data.withColumn("STATUS", when(data["STATUS"]== "X", -1).when(data["STATUS"] == "C", -2).otherwise(data["STATUS"]))

#casting the datatype to integer
data=data.withColumn("STATUS", data["STATUS"].cast(IntegerType()))

#Creating a new variable for classifying customer based on deliquency
data=data.withColumn("DELIQUENT", when(data["STATUS"]== 2, 1).when(data["STATUS"]>2, 1).otherwise(0))

#dropping the status column
data=data.drop("STATUS")

In [59]:
#dropping flag_mobil column, since it reductant
data=data.drop("FLAG_MOBIL")

### Binning numerical columns in data

In [123]:
#converting dataframe to pandas dataframe
dt=data.toPandas()

In [124]:
#Binning CNT_CHILDREN variable
dt.isnull().sum()

ID                     0
CODE_GENDER            0
FLAG_OWN_CAR           0
FLAG_OWN_REALTY        0
CNT_CHILDREN           0
NAME_INCOME_TYPE       0
NAME_EDUCATION_TYPE    0
NAME_FAMILY_STATUS     0
NAME_HOUSING_TYPE      0
FLAG_WORK_PHONE        0
FLAG_PHONE             0
FLAG_EMAIL             0
OCCUPATION_TYPE        0
CNT_FAM_MEMBERS        0
AGE                    0
YEARS_EMPLOYED         0
MONTHLY_INCOME         0
DELIQUENT              0
dtype: int64

In [125]:
#Binning CNT_CHILDREN variable
dt["CHILDREN"]=pd.cut(dt.CNT_CHILDREN, bins=[0,1,2,4,19],labels=["0",'1','2-3',"> 4"], right=False)
dt["CHILDREN"].value_counts()

0      51927
1      15167
2-3     7609
> 4      170
Name: CHILDREN, dtype: int64

Each category should have 5% of total counts. So the last category has to be dropped.

In [126]:
#Binning MONTHLY_INCOME variable
dt["SALARY"]=pd.qcut(dt.MONTHLY_INCOME, q=5, labels=["lowest","very low", "low","moderate", "high"])
dt["SALARY"].value_counts()

lowest      17783
low         15960
moderate    15312
high        13607
very low    12212
Name: SALARY, dtype: int64

In [127]:
#Binning CNT_FAM_MEMBERS variable
dt["FAMILY_MEM"]=pd.cut(dt.CNT_FAM_MEMBERS, bins=[1,2,4,7,20],labels=['1','<=3','<=6', '>6'],right=False)
dt["FAMILY_MEM"].value_counts()

<=3    53173
1      14292
<=6     7360
>6        48
Name: FAMILY_MEM, dtype: int64

Each category should have 5% of total counts. So the last category has to be dropped.

In [128]:
#Binning AGE variable
dt["AGE_RANGE"]=pd.qcut(dt.AGE, q=5, labels=["<=33","<=39","<=46","<=55",">55"])
dt["AGE_RANGE"].value_counts()

<=33    16294
<=39    15112
<=55    15024
>55     14446
<=46    13998
Name: AGE_RANGE, dtype: int64

In [129]:
#Binning YEARS_EMPLOYED variable
dt["WORK_LEVEL"]=pd.cut(dt.YEARS_EMPLOYED, bins=[-999,0,2,4,6,9,47],labels=["retired","fresher","intermediate","first-level","mid-level","senior-level"],right=False)
dt["WORK_LEVEL"].value_counts()

senior-level    17465
retired         12617
fresher         11689
mid-level       11589
intermediate    11426
first-level     10088
Name: WORK_LEVEL, dtype: int64

In [130]:
#checking for null values
dt.isnull().sum()

ID                     0
CODE_GENDER            0
FLAG_OWN_CAR           0
FLAG_OWN_REALTY        0
CNT_CHILDREN           0
NAME_INCOME_TYPE       0
NAME_EDUCATION_TYPE    0
NAME_FAMILY_STATUS     0
NAME_HOUSING_TYPE      0
FLAG_WORK_PHONE        0
FLAG_PHONE             0
FLAG_EMAIL             0
OCCUPATION_TYPE        0
CNT_FAM_MEMBERS        0
AGE                    0
YEARS_EMPLOYED         0
MONTHLY_INCOME         0
DELIQUENT              0
CHILDREN               1
SALARY                 0
FAMILY_MEM             1
AGE_RANGE              0
WORK_LEVEL             0
dtype: int64

In [131]:
#dropping null values
new_data = dt.dropna(axis = 0, how ='any')

In [132]:
#Changong the data types

new_data[["CHILDREN"]] = new_data[["CHILDREN"]].astype('str')
new_data[["SALARY"]] =new_data[["SALARY"]].astype('str')
new_data[["FAMILY_MEM"]]=new_data[["FAMILY_MEM"]].astype('str')
new_data[["AGE_RANGE"]]=new_data[["AGE_RANGE"]].astype('str')
new_data[["WORK_LEVEL"]]=new_data[["WORK_LEVEL"]].astype('str')
new_data[["DELIQUENT"]]=new_data[["DELIQUENT"]].astype('str')

/home/ec2-user/.local/lib/python3.7/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [133]:
#Dropping parent column and ID column
dataset=new_data.drop(["ID","CNT_CHILDREN","MONTHLY_INCOME","CNT_FAM_MEMBERS","AGE","YEARS_EMPLOYED"], axis=1)

In [235]:
#Converting to spark dataframe

df=spark.createDataFrame(dataset)
df.printSchema()

root
 |-- CODE_GENDER: string (nullable = true)
 |-- FLAG_OWN_CAR: string (nullable = true)
 |-- FLAG_OWN_REALTY: string (nullable = true)
 |-- NAME_INCOME_TYPE: string (nullable = true)
 |-- NAME_EDUCATION_TYPE: string (nullable = true)
 |-- NAME_FAMILY_STATUS: string (nullable = true)
 |-- NAME_HOUSING_TYPE: string (nullable = true)
 |-- FLAG_WORK_PHONE: long (nullable = true)
 |-- FLAG_PHONE: long (nullable = true)
 |-- FLAG_EMAIL: long (nullable = true)
 |-- OCCUPATION_TYPE: string (nullable = true)
 |-- DELIQUENT: string (nullable = true)
 |-- CHILDREN: string (nullable = true)
 |-- SALARY: string (nullable = true)
 |-- FAMILY_MEM: string (nullable = true)
 |-- AGE_RANGE: string (nullable = true)
 |-- WORK_LEVEL: string (nullable = true)



In [236]:
#dropping categories with lewss than 5%
df=df.filter("not (CHILDREN == '> 4')")

In [237]:
#checking wheter the filterig of children variable have mended the family_mem variable
df.groupby(["FAMILY_MEM"]).count().sort(desc("count")).show()

21/07/21 12:20:05 WARN TaskSetManager: Stage 2670 contains a task of very large size (7995 KB). The maximum recommended task size is 100 KB.


+----------+-----+
|FAMILY_MEM|count|
+----------+-----+
|       <=3|53173|
|         1|14292|
|       <=6| 7238|
+----------+-----+



Filtering of children column have taken care of family_mem column too

### Weight of Evidence and Information Value Class

In [238]:
class WOE_IV(object):
    
    """
    This class contains methods that are used for calculating the WOE and IV values in a given dataset.
    
    Attributes
    ----------
    df : spark DataFrame
        The spark dataframe
    cols_to_woe : list
        Columns that you want to convert in WOE and IV
    label_column : str
        target column of the data
    good_label : str
        Predicted variable in the form of 1 or 0
    """
    
    def __init__(self, df: DataFrame, cols_to_woe: [str], label_column: str, good_label: str):
        self.df = df
        self.cols_to_woe = cols_to_woe
        self.label_column = label_column
        self.good_label = good_label
        self.fit_data = {} # This is a dictionary which will store all the WOE and IV computed in the latter part of the code

    def fit(self):  
        """
        This method takes the columns in 'cols_to_woe' and calculates the good and bad amounts, and then assigns the WOE values in the dictionary, 'fit_data'
        """
        for col_to_woe in self.cols_to_woe: # Iterating over each column in the list, cols_to_woe
            
            total_good = self.compute_total_amount_of_good() # Computes the total amount of good labels in the entire dataset
            total_bad = self.compute_total_amount_of_bad() # Computes the total amount of bad labels in the entire dataset
            
            woe_df = self.df.select(col_to_woe) # This dataframe stores the column, col_to_woe from the DataFrame
            categories = woe_df.distinct().collect() # Creates a list of all the distinct categories in the column, col_to_woe
            for category_row in categories: # Iterating over each category in the distinct categories of the column
                category = category_row[col_to_woe]
                
                # For each category, we calculate the good_amount and bad_amount
                good_amount = self.compute_good_amount(col_to_woe, category)
                bad_amount = self.compute_bad_amount(col_to_woe, category)
                
                # If the good_amount or bad_amount is 0, we replace it with 0.5, because in the calculation of woe, it comes in the denominator resulting in a Nan value which will give an error
                good_amount = good_amount if good_amount != 0 else 0.5
                bad_amount = bad_amount if bad_amount != 0 else 0.5

                # Calculating the fraction of good_amount and bad_amount specific to that category
                good_dist = good_amount / total_good
                bad_dist = bad_amount / total_bad

                # Calling the build_fit_data method which calculates the WOE and IV for the category in the column and stores it in the dictionary, fit_data
                self.build_fit_data(col_to_woe, category, good_dist, bad_dist)

    def transform(self, df: DataFrame):
        """
        This method creates a column called "col_to_woe + '_woe'" which replaces the given category value with the WOE value.
        It finally returns a final dataframe containing the WOE values in place of the columns
        """
        def _encode_woe(col_to_woe_):
            return coalesce(
                *[when(col(col_to_woe_) == category, lit(woe_iv['woe']))
                  for category, woe_iv in self.fit_data[col_to_woe_].items()]
            )

        for col_to_woe, woe_info in self.fit_data.items():
            # Adds another column to the df containing the final WOE values
            df = df.withColumn(col_to_woe + '_woe', _encode_woe(col_to_woe))
        return df

    def compute_total_amount_of_good(self):
     # Computes the total amount of good labels in the entire dataset
        return self.df.select(self.label_column).filter(col(self.label_column) == self.good_label).count()

    def compute_total_amount_of_bad(self):
     # Computes the total amount of bad labels in the entire dataset
        return self.df.select(self.label_column).filter(col(self.label_column) != self.good_label).count()

    def compute_good_amount(self, col_to_woe: str, category: str):
     # Computes the total amount of good labels for the specific class in the feature
        return self.df.select(col_to_woe, self.label_column)\
                      .filter(
                            (col(col_to_woe) == category) & (col(self.label_column) == self.good_label)
                      ).count()

    def compute_bad_amount(self, col_to_woe: str, category: str):
        # Computes the total amount of bad labels for the specific class in the feature
        return self.df.select(col_to_woe, self.label_column)\
                      .filter(
                            (col(col_to_woe) == category) & (col(self.label_column) != self.good_label)
                      ).count()

    def build_fit_data(self, col_to_woe, category, good_dist, bad_dist):
        """
        Calculates the WOE and value for the specific category and saves it in the woe_info dictionary which is saves in the fit_data dictionary
        
        
        Attributes:
        col_to_woe: str
        name of the feature
        
        category: str
        Class of that feature
        
        good_dist: float
        Fraction of good labels specific to that category
        
        bad_dist: float
        Fraction of bad labels specific to that category
        """
        woe_info = {
            category: {
                'woe': math.log(good_dist / bad_dist), # Calculating the WOE value for the category in the column
                'iv': (good_dist - bad_dist) * math.log(good_dist / bad_dist) # Calculating the IV value for the category in the column
            }
        }

        # Adding the feature, col_to_woe to fit_data if not present already
        if col_to_woe not in self.fit_data:
            self.fit_data[col_to_woe] = woe_info
        else:
            self.fit_data[col_to_woe].update(woe_info)

    def compute_iv(self):
        """
        Returns the IV dictionary which contains the IV value for each feature
        """
        iv_dict = {}

        for woe_col, categories in self.fit_data.items():
            iv_dict[woe_col] = 0
            for category, woe_iv in categories.items():
                iv_dict[woe_col] += woe_iv['iv'] # IV for the feature = IV value for the indiviual categories of the feature (column)
        return iv_dict

In [239]:
#list of columns
cCols=df.columns

#removing the target column
cCols.remove("DELIQUENT")

In [240]:
#Decalaring the class
woe=WOE_IV(df, cCols, "DELIQUENT", "0")

In [241]:
#fittonmg the class
woe.fit()

21/07/21 12:20:06 WARN TaskSetManager: Stage 2672 contains a task of very large size (7995 KB). The maximum recommended task size is 100 KB.
21/07/21 12:20:06 WARN TaskSetManager: Stage 2674 contains a task of very large size (7995 KB). The maximum recommended task size is 100 KB.
21/07/21 12:20:06 WARN TaskSetManager: Stage 2676 contains a task of very large size (7995 KB). The maximum recommended task size is 100 KB.
21/07/21 12:20:07 WARN TaskSetManager: Stage 2678 contains a task of very large size (7995 KB). The maximum recommended task size is 100 KB.
21/07/21 12:20:08 WARN TaskSetManager: Stage 2680 contains a task of very large size (7995 KB). The maximum recommended task size is 100 KB.
21/07/21 12:20:08 WARN TaskSetManager: Stage 2682 contains a task of very large size (7995 KB). The maximum recommended task size is 100 KB.
21/07/21 12:20:09 WARN TaskSetManager: Stage 2684 contains a task of very large size (7995 KB). The maximum recommended task size is 100 KB.
21/07/21 12:2

In [242]:
#transforming the dtaset to woe
final_df=woe.transform(df)

In [244]:
#schema of the final dataset
final_df.printSchema()

root
 |-- CODE_GENDER: string (nullable = true)
 |-- FLAG_OWN_CAR: string (nullable = true)
 |-- FLAG_OWN_REALTY: string (nullable = true)
 |-- NAME_INCOME_TYPE: string (nullable = true)
 |-- NAME_EDUCATION_TYPE: string (nullable = true)
 |-- NAME_FAMILY_STATUS: string (nullable = true)
 |-- NAME_HOUSING_TYPE: string (nullable = true)
 |-- FLAG_WORK_PHONE: long (nullable = true)
 |-- FLAG_PHONE: long (nullable = true)
 |-- FLAG_EMAIL: long (nullable = true)
 |-- OCCUPATION_TYPE: string (nullable = true)
 |-- DELIQUENT: string (nullable = true)
 |-- CHILDREN: string (nullable = true)
 |-- SALARY: string (nullable = true)
 |-- FAMILY_MEM: string (nullable = true)
 |-- AGE_RANGE: string (nullable = true)
 |-- WORK_LEVEL: string (nullable = true)
 |-- CODE_GENDER_woe: double (nullable = true)
 |-- FLAG_OWN_CAR_woe: double (nullable = true)
 |-- FLAG_OWN_REALTY_woe: double (nullable = true)
 |-- NAME_INCOME_TYPE_woe: double (nullable = true)
 |-- NAME_EDUCATION_TYPE_woe: double (nullable = 

In [245]:
#dropping the parent columns
final_df=final_df.drop('CODE_GENDER','FLAG_OWN_CAR','FLAG_OWN_REALTY','NAME_INCOME_TYPE','NAME_EDUCATION_TYPE','NAME_FAMILY_STATUS','NAME_HOUSING_TYPE','FLAG_WORK_PHONE','FLAG_PHONE','FLAG_EMAIL','OCCUPATION_TYPE','CHILDREN','SALARY','FAMILY_MEM','AGE_RANGE','WORK_LEVEL')
final_df.printSchema()

root
 |-- DELIQUENT: string (nullable = true)
 |-- CODE_GENDER_woe: double (nullable = true)
 |-- FLAG_OWN_CAR_woe: double (nullable = true)
 |-- FLAG_OWN_REALTY_woe: double (nullable = true)
 |-- NAME_INCOME_TYPE_woe: double (nullable = true)
 |-- NAME_EDUCATION_TYPE_woe: double (nullable = true)
 |-- NAME_FAMILY_STATUS_woe: double (nullable = true)
 |-- NAME_HOUSING_TYPE_woe: double (nullable = true)
 |-- FLAG_WORK_PHONE_woe: double (nullable = true)
 |-- FLAG_PHONE_woe: double (nullable = true)
 |-- FLAG_EMAIL_woe: double (nullable = true)
 |-- OCCUPATION_TYPE_woe: double (nullable = true)
 |-- CHILDREN_woe: double (nullable = true)
 |-- SALARY_woe: double (nullable = true)
 |-- FAMILY_MEM_woe: double (nullable = true)
 |-- AGE_RANGE_woe: double (nullable = true)
 |-- WORK_LEVEL_woe: double (nullable = true)



In [247]:
#computing the IV values
ivs=woe.compute_iv()

In [248]:
#changing the data type of target variable to integer
final_df= final_df.withColumn('DELIQUENT', final_df['DELIQUENT'].cast(IntegerType()))
final_df.printSchema()

root
 |-- DELIQUENT: integer (nullable = true)
 |-- CODE_GENDER_woe: double (nullable = true)
 |-- FLAG_OWN_CAR_woe: double (nullable = true)
 |-- FLAG_OWN_REALTY_woe: double (nullable = true)
 |-- NAME_INCOME_TYPE_woe: double (nullable = true)
 |-- NAME_EDUCATION_TYPE_woe: double (nullable = true)
 |-- NAME_FAMILY_STATUS_woe: double (nullable = true)
 |-- NAME_HOUSING_TYPE_woe: double (nullable = true)
 |-- FLAG_WORK_PHONE_woe: double (nullable = true)
 |-- FLAG_PHONE_woe: double (nullable = true)
 |-- FLAG_EMAIL_woe: double (nullable = true)
 |-- OCCUPATION_TYPE_woe: double (nullable = true)
 |-- CHILDREN_woe: double (nullable = true)
 |-- SALARY_woe: double (nullable = true)
 |-- FAMILY_MEM_woe: double (nullable = true)
 |-- AGE_RANGE_woe: double (nullable = true)
 |-- WORK_LEVEL_woe: double (nullable = true)



## Step 4: Model Building

In [324]:
#importing necessary libraries
from pyspark.ml.feature import VectorAssembler

trainCols=final_df.columns
trainCols.remove("DELIQUENT")

#configure the assembler object and use it to transform df
assembler = VectorAssembler(inputCols=trainCols, outputCol='features')
m_df = assembler.transform(final_df)

In [325]:
#train test spplit
m_train, m_test = m_df.randomSplit([0.7,0.3], seed=2018)

In [326]:
# train model in the training dataset

# import the logistic model
from pyspark.ml.classification import LogisticRegression

# create the logistic model, configure it and call fit it to the data. 
lr = LogisticRegression(featuresCol='features', labelCol='DELIQUENT')
lrModel = lr.fit(m_train)

21/07/21 14:22:48 WARN TaskSetManager: Stage 3287 contains a task of very large size (7995 KB). The maximum recommended task size is 100 KB.
21/07/21 14:22:49 WARN TaskSetManager: Stage 3288 contains a task of very large size (7995 KB). The maximum recommended task size is 100 KB.
21/07/21 14:22:49 WARN TaskSetManager: Stage 3289 contains a task of very large size (7995 KB). The maximum recommended task size is 100 KB.
21/07/21 14:22:49 WARN TaskSetManager: Stage 3290 contains a task of very large size (7995 KB). The maximum recommended task size is 100 KB.
21/07/21 14:22:50 WARN TaskSetManager: Stage 3291 contains a task of very large size (7995 KB). The maximum recommended task size is 100 KB.
21/07/21 14:22:50 WARN TaskSetManager: Stage 3292 contains a task of very large size (7995 KB). The maximum recommended task size is 100 KB.
21/07/21 14:22:50 WARN TaskSetManager: Stage 3293 contains a task of very large size (7995 KB). The maximum recommended task size is 100 KB.
21/07/21 14:2

In [327]:
# use the model to predict the class labels of test data. 
lrPredictions = lrModel.transform(m_test)

## Step 5 : Model Evaluation

In [328]:
#evaluating the model on test set
evals=lrModel.evaluate(m_test)

In [329]:
#printing the metrics
print("Recall : ", evals.recallByLabel)
print("Precision : ", evals.precisionByLabel)
print("Accuracy : ", evals.accuracy)
print("Area undre ROC : ", evals.areaUnderROC)

21/07/21 14:22:54 WARN TaskSetManager: Stage 3305 contains a task of very large size (7995 KB). The maximum recommended task size is 100 KB.
21/07/21 14:22:56 WARN TaskSetManager: Stage 3307 contains a task of very large size (7995 KB). The maximum recommended task size is 100 KB.
21/07/21 14:22:57 WARN TaskSetManager: Stage 3309 contains a task of very large size (7995 KB). The maximum recommended task size is 100 KB.


Recall :  [1.0, 0.0]


21/07/21 14:22:58 WARN TaskSetManager: Stage 3311 contains a task of very large size (7995 KB). The maximum recommended task size is 100 KB.


Precision :  [0.9842950571695511, 0.0]
Accuracy :  0.9842950571695511


[Stage 3311:>                                                       (0 + 2) / 2]

Area undre ROC :  0.6373329464675793


The metrics all point ot the imabalnced dataset.

### Feature Selection

According to the problem statement, the IV values of varaibles with threshold of 0.002 is considreed as weak predictors and need to be eliminated.

In [330]:
#IV values
ivs

{'CODE_GENDER': 0.015269334044836883,
 'FLAG_OWN_CAR': 0.00041878796811565925,
 'FLAG_OWN_REALTY': 0.023588684509955854,
 'NAME_INCOME_TYPE': 0.03573845068537383,
 'NAME_EDUCATION_TYPE': 0.01787406777867823,
 'NAME_FAMILY_STATUS': 0.03739668321149994,
 'NAME_HOUSING_TYPE': 0.03796014929496968,
 'FLAG_WORK_PHONE': 0.0015284588205042218,
 'FLAG_PHONE': 0.0015274389064683746,
 'FLAG_EMAIL': 0.0004914832435765251,
 'OCCUPATION_TYPE': 0.07269488836462261,
 'CHILDREN': 0.0016836041739970286,
 'SALARY': 0.00400612317786641,
 'FAMILY_MEM': 0.012840987465373172,
 'AGE_RANGE': 0.027831051283030477,
 'WORK_LEVEL': 0.07384595780538622}

In [331]:
#dropping weak predictors
final_df=final_df.drop('FLAG_OWN_CAR_woe','FLAG_WORK_PHONE_woe', 'FLAG_PHONE_woe','FLAG_EMAIL_woe', 'CHILDREN_woe', 'SALARY_woe')

In [332]:
#remodelling the model

trainCols=final_df.columns
trainCols.remove("DELIQUENT")

#configure the assembler object and use it to transform df
assembler = VectorAssembler(inputCols=trainCols, outputCol='features')
m_df = assembler.transform(final_df)

In [333]:
m_train, m_test = m_df.randomSplit([0.7,0.3], seed=2018)

In [334]:
#refiting the data
lr = LogisticRegression(featuresCol='features', labelCol='DELIQUENT')
lrModel = lr.fit(m_train)

21/07/21 14:32:10 WARN TaskSetManager: Stage 3325 contains a task of very large size (7995 KB). The maximum recommended task size is 100 KB.
21/07/21 14:32:11 WARN TaskSetManager: Stage 3326 contains a task of very large size (7995 KB). The maximum recommended task size is 100 KB.
21/07/21 14:32:11 WARN TaskSetManager: Stage 3327 contains a task of very large size (7995 KB). The maximum recommended task size is 100 KB.
21/07/21 14:32:11 WARN TaskSetManager: Stage 3328 contains a task of very large size (7995 KB). The maximum recommended task size is 100 KB.
21/07/21 14:32:12 WARN TaskSetManager: Stage 3329 contains a task of very large size (7995 KB). The maximum recommended task size is 100 KB.
21/07/21 14:32:12 WARN TaskSetManager: Stage 3330 contains a task of very large size (7995 KB). The maximum recommended task size is 100 KB.
21/07/21 14:32:12 WARN TaskSetManager: Stage 3331 contains a task of very large size (7995 KB). The maximum recommended task size is 100 KB.
21/07/21 14:3

In [335]:
# use the model to predict the class labels of test data. 
lrPredictions = lrModel.transform(m_test)

In [336]:
evals=lrModel.evaluate(m_test)

In [337]:
print("Recall : ", evals.recallByLabel)
print("Precision : ", evals.precisionByLabel)
print("Accuracy : ", evals.accuracy)
print("Area undre ROC : ", evals.areaUnderROC)

21/07/21 14:33:33 WARN TaskSetManager: Stage 3339 contains a task of very large size (7995 KB). The maximum recommended task size is 100 KB.
21/07/21 14:33:34 WARN TaskSetManager: Stage 3341 contains a task of very large size (7995 KB). The maximum recommended task size is 100 KB.
21/07/21 14:33:35 WARN TaskSetManager: Stage 3343 contains a task of very large size (7995 KB). The maximum recommended task size is 100 KB.


Recall :  [1.0, 0.0]


Precision :  [0.9842950571695511, 0.0]
Accuracy :  0.9842950571695511


21/07/21 14:33:35 WARN TaskSetManager: Stage 3345 contains a task of very large size (7995 KB). The maximum recommended task size is 100 KB.
[Stage 3345:>                                                       (0 + 2) / 2]

Area undre ROC :  0.6234276749692563


### Undersampling the data because of its imbalance

In [338]:
#to undersample that dataframe
major_df = final_df.filter(col("DELIQUENT") == 0)
minor_df = final_df.filter(col("DELIQUENT") == 1)
ratio = int(major_df.count()/minor_df.count())
print("ratio: {}".format(ratio))

sampled_majority_df = major_df.sample(False, 1/ratio)
combined_df_2 = sampled_majority_df.unionAll(minor_df)

21/07/21 14:38:02 WARN TaskSetManager: Stage 3359 contains a task of very large size (7995 KB). The maximum recommended task size is 100 KB.
21/07/21 14:38:02 WARN TaskSetManager: Stage 3361 contains a task of very large size (7995 KB). The maximum recommended task size is 100 KB.


ratio: 63


In [339]:
#reconfigure the assembler object and use it to transform df
assembler = VectorAssembler(inputCols=trainCols, outputCol='features')
m_df = assembler.transform(combined_df_2)

In [340]:
m_train, m_test = m_df.randomSplit([0.7,0.3], seed=2018)

In [341]:
# train model in the training dataset

# create the logistic model, configure it and call fit it to the data. 
lr = LogisticRegression(featuresCol='features', labelCol='DELIQUENT')
lrModel = lr.fit(m_train)

21/07/21 14:38:34 WARN TaskSetManager: Stage 3363 contains a task of very large size (7995 KB). The maximum recommended task size is 100 KB.
21/07/21 14:38:35 WARN TaskSetManager: Stage 3364 contains a task of very large size (7995 KB). The maximum recommended task size is 100 KB.
21/07/21 14:38:35 WARN TaskSetManager: Stage 3365 contains a task of very large size (7995 KB). The maximum recommended task size is 100 KB.
21/07/21 14:38:36 WARN TaskSetManager: Stage 3366 contains a task of very large size (7995 KB). The maximum recommended task size is 100 KB.
21/07/21 14:38:36 WARN TaskSetManager: Stage 3367 contains a task of very large size (7995 KB). The maximum recommended task size is 100 KB.
21/07/21 14:38:36 WARN TaskSetManager: Stage 3368 contains a task of very large size (7995 KB). The maximum recommended task size is 100 KB.
21/07/21 14:38:36 WARN TaskSetManager: Stage 3369 contains a task of very large size (7995 KB). The maximum recommended task size is 100 KB.
21/07/21 14:3

In [342]:
# use the model to predict the class labels of test data. 
lrPredictions = lrModel.transform(m_test)

In [343]:
evals=lrModel.evaluate(m_test)

In [344]:
print("Recall : ", evals.recallByLabel)
print("Precision : ", evals.precisionByLabel)
print("Accuracy : ", evals.accuracy)
print("Area undre ROC : ", evals.areaUnderROC)

21/07/21 14:39:18 WARN TaskSetManager: Stage 3376 contains a task of very large size (7995 KB). The maximum recommended task size is 100 KB.
21/07/21 14:39:19 WARN TaskSetManager: Stage 3378 contains a task of very large size (7995 KB). The maximum recommended task size is 100 KB.
21/07/21 14:39:19 WARN TaskSetManager: Stage 3380 contains a task of very large size (7995 KB). The maximum recommended task size is 100 KB.


Recall :  [0.5393258426966292, 0.5661971830985916]


21/07/21 14:39:20 WARN TaskSetManager: Stage 3382 contains a task of very large size (7995 KB). The maximum recommended task size is 100 KB.


Precision :  [0.5549132947976878, 0.5506849315068493]
Accuracy :  0.5527426160337553


Area undre ROC :  0.5970367146700427
